# Create LoadData CSVs with the paths to IC functions for analysis

In this notebook, we create LoadData CSVs that contains paths to each channel per image set and associated illumination correction `npy` files per channel for CellProfiler to process. 

## Import libraries

In [1]:
import pathlib
import pandas as pd
import re

import sys

sys.path.append("../utils")
import loaddata_utils as ld_utils

## Set paths

In [2]:
# Paths for parameters to make loaddata csv
index_directory = pathlib.Path("/media/18tbdrive/ALSF_pilot_data/SN0313537/")
config_dir_path = pathlib.Path("../1.illumination_correction/config_files").absolute()
output_csv_dir = pathlib.Path("./loaddata_csvs")
output_csv_dir.mkdir(parents=True, exist_ok=True)
illum_directory = pathlib.Path("../1.illumination_correction/illum_directory")

# Find all 'Images' folders within the directory
images_folders = list(index_directory.rglob('Images'))

## Create LoadData CSVs with illumination functions for all data

In [3]:
# Define the default config path for BR00 folders
default_config_path = pathlib.Path(f"{config_dir_path}/config.yml")

# Loop through each folder and create a LoadData CSV
for folder in images_folders:
    # Get the relative path of the folder and the first-level folder name
    relative_path = folder.relative_to(index_directory)
    first_folder = relative_path.parts[0]  # First-level folder

    # Initialize plate_name and config_path
    plate_name = None
    config_path = None

    # For BR00 folders, use the default config.yml
    if first_folder.startswith("BR00"):
        plate_name = first_folder.split("_")[0]  # Take the first part as plate_name
        config_path = default_config_path

    # For 2024 folders, locate a matching config file
    elif first_folder.startswith("2024"):
        second_folder = relative_path.parts[1]
        part1 = "_".join(first_folder.split("_")[-2:])  # Last two parts of first folder
        part2 = second_folder.split("_")[0]  # First part of second folder
        plate_name = f"{part1}_{part2}"

        # Search for a matching config file for the 2024 folder
        matching_configs = list(config_dir_path.glob(f"{part1.split('_')[0]}_*.yml"))
        if matching_configs:
            config_path = matching_configs[0]
        else:
            print(f"No matching config file for 2024 plate: {plate_name}")
            continue  # Skip if no matching config file

    else:
        print(f"Unexpected folder pattern: {folder}")
        continue

    # Print the path of the config file being used
    print(f"Config path for plate '{plate_name}': {config_path}")

    # Proceed with creating LoadData CSV if config_path is defined
    plate_id = next(part for part in plate_name.split("_") if part.startswith("BR00"))
    illum_output_path = (
        pathlib.Path(illum_directory / plate_id).absolute().resolve(strict=True)
    )
    path_to_output_csv = (
        output_csv_dir / f"{plate_name}_loaddata_original.csv"
    ).absolute()
    path_to_output_with_illum_csv = path_to_output_csv.with_name(
        path_to_output_csv.name.replace("original", "with_illum")
    )

    # Call the function to create the LoadData CSV
    ld_utils.create_loaddata_illum_csv(
        index_directory=folder,
        config_path=config_path,
        path_to_output=path_to_output_csv,
        illum_directory=illum_output_path,
        plate_id=plate_id,
        illum_output_path=path_to_output_with_illum_csv,
    )

Config path for plate 'SK-N-MC_Re-imaged_BR00143976': /home/jenna/pediatric_cancer_atlas_profiling/2.feature_extraction/../1.illumination_correction/config_files/SK-N-MC_config.yml
SK-N-MC_Re-imaged_BR00143976_loaddata_with_illum.csv is created!
The SK-N-MC_Re-imaged_BR00143976_loaddata_original.csv CSV file has been removed as it does not contain the IC functions.
Config path for plate 'SK-N-MC_Re-imaged_BR00143978': /home/jenna/pediatric_cancer_atlas_profiling/2.feature_extraction/../1.illumination_correction/config_files/SK-N-MC_config.yml
SK-N-MC_Re-imaged_BR00143978_loaddata_with_illum.csv is created!
The SK-N-MC_Re-imaged_BR00143978_loaddata_original.csv CSV file has been removed as it does not contain the IC functions.
Config path for plate 'SK-N-MC_Re-imaged_BR00143980': /home/jenna/pediatric_cancer_atlas_profiling/2.feature_extraction/../1.illumination_correction/config_files/SK-N-MC_config.yml
SK-N-MC_Re-imaged_BR00143980_loaddata_with_illum.csv is created!
The SK-N-MC_Re-ima

## Concat the re-imaged data back to their original plate and remove the original poor quality data paths

All CSVs have linkage back to the IC directory and functions.

### Collect a list of original CSVs and identify unique plate IDs

In [4]:
# Step 1: Find all CSV files in the output directory
csv_files = list(output_csv_dir.glob("*.csv"))

# Step 2: Extract unique BR00 IDs from filenames
br00_pattern = re.compile(r"(BR00\d+)")  # Regex to match 'BR00' followed by digits

# Collect all matching BR00 IDs from filenames
br00_ids = {br00_pattern.search(csv_file.stem).group(1) 
            for csv_file in csv_files 
            if br00_pattern.search(csv_file.stem)}

# Sort BR00 IDs numerically to be consistent in ordering
br00_ids = sorted(br00_ids, key=lambda x: int(x[4:]))  # Convert digits part to integer for numerical sorting

print(f"Found {len(br00_ids)} BR00 IDs: {br00_ids}")

Found 6 BR00 IDs: ['BR00143976', 'BR00143977', 'BR00143978', 'BR00143979', 'BR00143980', 'BR00143981']


### Track/store files and add a metadata column for if a row is re-imaged or not

In [5]:
# Step 3: Initialize storage to track used files and find proper column order 
br00_dataframes = {br_id: [] for br_id in br00_ids}
used_files = set()  # Store filenames used in the process
concat_files = []  # Track new concatenated CSV files

# Load one BR00 starting CSV that will have the correct column order
column_order = pd.read_csv(pathlib.Path(f"{output_csv_dir}/{list(br00_ids)[0]}_loaddata_with_illum.csv"), nrows=0).columns.tolist()

# Step 4: Add 'Metadata_Reimaged' column and group by BR00 ID
for csv_file in csv_files:
    filename = csv_file.stem
    match = br00_pattern.search(filename)

    if match:
        br_id = match.group(1)

        # Read the CSV file into a DataFrame
        loaddata_df = pd.read_csv(csv_file)

        # Reorder DataFrame columns to match the correct column order
        loaddata_df = loaddata_df[column_order]  # Ensure the columns are in the correct order

        # Add 'Metadata_Reimaged' column based on filename
        loaddata_df['Metadata_Reimaged'] = 'Re-imaged' in filename

        # Append the DataFrame to the corresponding BR00 group
        br00_dataframes[br_id].append(loaddata_df)

        # Track this file as used
        used_files.add(csv_file.name)

# Print an example DataFrame (first BR00 group)
example_id = next(iter(br00_dataframes))  # Get the first BR00 ID
example_df = pd.concat(br00_dataframes[example_id], ignore_index=True)
print(f"\nExample DataFrame for BR00 ID: {example_id}")
example_df.iloc[:, [0, 1, -1]] # Display only the first two and last column


Example DataFrame for BR00 ID: BR00143976


,FileName_OrigBrightfield,PathName_OrigBrightfield,Metadata_Reimaged
0,r03c03f01p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/BR0...,False
1,r03c03f02p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/BR0...,False
2,r03c03f03p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/BR0...,False
3,r03c03f04p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/BR0...,False
4,r03c03f05p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/BR0...,False
...,...,...,...
2842,r14c12f05p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/202...,True
2843,r14c12f06p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/202...,True
2844,r14c12f07p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/202...,True
2845,r14c12f08p01-ch1sk1fk1fl1.tiff,/media/18tbdrive/ALSF_pilot_data/SN0313537/202...,True


### Concat the re-imaged and original data for the same plate and remove any duplicate wells that come from the original data

We remove the duplicates that aren't re-imaged since they are of poor quality. We want to analyze the re-imaged data from those same wells.

In [6]:
# Step 5: Concatenate DataFrames, drop duplicates, and save per BR00 ID
for br_id, dfs in br00_dataframes.items():
    if dfs:  # Only process if there are matching files
        concatenated_df = pd.concat(dfs, ignore_index=True)

        # Drop duplicates, prioritizing rows with 'Metadata_Reimaged' == True
        deduplicated_df = concatenated_df.sort_values(
            'Metadata_Reimaged', ascending=False
        ).drop_duplicates(subset=['Metadata_Well', 'Metadata_Site'], keep='first')

        # Sort by 'Metadata_Col', 'Metadata_Row', and 'Metadata_Site
        sorted_df = deduplicated_df.sort_values(
            ['Metadata_Col', 'Metadata_Row', "Metadata_Site"], ascending=True
        )

        # Save the cleaned, concatenated, and sorted DataFrame to a new CSV file
        output_path = output_csv_dir / f"{br_id}_concatenated_with_illum.csv"
        sorted_df.to_csv(output_path, index=False)

        print(f"Saved: {output_path}")
        concat_files.append(output_path)  # Track new concatenated files
    else:
        print(f"No files found for {br_id}")

Saved: loaddata_csvs/BR00143976_concatenated_with_illum.csv
Saved: loaddata_csvs/BR00143977_concatenated_with_illum.csv
Saved: loaddata_csvs/BR00143978_concatenated_with_illum.csv
Saved: loaddata_csvs/BR00143979_concatenated_with_illum.csv
Saved: loaddata_csvs/BR00143980_concatenated_with_illum.csv
Saved: loaddata_csvs/BR00143981_concatenated_with_illum.csv


### Confirm that all LoadData CSV files were included in previous concat (avoid data loss)

In [7]:
# Step 6: Verify all files were used
unused_files = set(csv_file.name for csv_file in csv_files) - used_files

if unused_files:
    print("Warning: Some files were not used in the concatenation!")
    for file in unused_files:
        print(f"Unused: {file}")
else:
    print("All files were successfully used.")

All files were successfully used.


### Remove the original CSV files to prevent CellProfiler from using them

In [8]:
# Step 7: Remove all non-concatenated CSVs to avoid confusion
for csv_file in csv_files:
    if csv_file not in concat_files:  # Keep only new concatenated files
        csv_file.unlink()  # Delete the file
        print(f"Removed: {csv_file}")

Removed: loaddata_csvs/SH-SY5Y_Re-imaged_BR00143978_loaddata_with_illum.csv
Removed: loaddata_csvs/NB-1_Re-imaged_BR00143977_loaddata_with_illum.csv
Removed: loaddata_csvs/KNS-42_Re-imaged_BR00143977_loaddata_with_illum.csv
Removed: loaddata_csvs/A-673_Re-imaged_BR00143978_loaddata_with_illum.csv
Removed: loaddata_csvs/BR00143978_loaddata_with_illum.csv
Removed: loaddata_csvs/KNS-42_Re-imaged_BR00143981_loaddata_with_illum.csv
Removed: loaddata_csvs/BR00143981_loaddata_with_illum.csv
Removed: loaddata_csvs/BR00143976_loaddata_with_illum.csv
Removed: loaddata_csvs/SH-SY5Y_Re-imaged_BR00143980_loaddata_with_illum.csv
Removed: loaddata_csvs/BR00143980_loaddata_with_illum.csv
Removed: loaddata_csvs/SK-N-MC_Re-imaged_BR00143978_loaddata_with_illum.csv
Removed: loaddata_csvs/BR00143979_loaddata_with_illum.csv
Removed: loaddata_csvs/SK-N-MC_Re-imaged_BR00143976_loaddata_with_illum.csv
Removed: loaddata_csvs/KP-N-YN_Re-imaged_BR00143977_loaddata_with_illum.csv
Removed: loaddata_csvs/CHP-212_Re